<a href="https://colab.research.google.com/github/vysf/machine-learning-practice/blob/master/Multiclass_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv('imdb_indonesian_movies_2.csv')

In [ ]:
df.head()

,judul_film,ringkasan_sinopsis,genre
0,Sunan Kalijaga,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Gie,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL Movie,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Sang pencerah,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [ ]:
df = df.drop(columns=['judul_film'])

In [ ]:
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


## One-Hot-Encoding

In [ ]:
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns=['genre'])
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


In [ ]:
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

In [ ]:
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

num_words = 500
tokenizer = Tokenizer(num_words=num_words, oov_token='x')

tokenizer.fit_on_texts(sinopsis_latih)
tokenizer.fit_on_texts(sinopsis_test)

sequens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sequens_test = tokenizer.texts_to_sequences(sinopsis_test)

padded_latih = pad_sequences(sequens_latih)
padded_test = pad_sequences(sequens_test)

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
num_epochs = 30
history = model.fit(
    padded_latih,
    label_latih,
    epochs=num_epochs,
    validation_data=(padded_test, label_test),
    verbose=2
)

Epoch 1/30
26/26 - 27s - loss: 1.6104 - accuracy: 0.1878 - val_loss: 1.6098 - val_accuracy: 0.1841 - 27s/epoch - 1s/step
Epoch 2/30
26/26 - 25s - loss: 1.6094 - accuracy: 0.1940 - val_loss: 1.6099 - val_accuracy: 0.1841 - 25s/epoch - 971ms/step
Epoch 3/30
26/26 - 24s - loss: 1.6083 - accuracy: 0.2575 - val_loss: 1.6088 - val_accuracy: 0.1990 - 24s/epoch - 906ms/step
Epoch 4/30
26/26 - 24s - loss: 1.6018 - accuracy: 0.2475 - val_loss: 1.6057 - val_accuracy: 0.2338 - 24s/epoch - 916ms/step
Epoch 5/30
26/26 - 24s - loss: 1.5691 - accuracy: 0.3010 - val_loss: 1.6005 - val_accuracy: 0.2239 - 24s/epoch - 912ms/step
Epoch 6/30
26/26 - 23s - loss: 1.4910 - accuracy: 0.3284 - val_loss: 1.6333 - val_accuracy: 0.2537 - 23s/epoch - 866ms/step
Epoch 7/30
26/26 - 24s - loss: 1.3640 - accuracy: 0.3980 - val_loss: 1.7971 - val_accuracy: 0.1891 - 24s/epoch - 915ms/step
Epoch 8/30
26/26 - 24s - loss: 1.2733 - accuracy: 0.4216 - val_loss: 1.8210 - val_accuracy: 0.2388 - 24s/epoch - 924ms/step
Epoch 9/30


In [ ]:
import numpy as np

labels = {
    0: "Drama",
    1: "Horor",
    2: "Komedi",
    3: "Laga",
    4: "Romantis",
}

# # set the input
input = """Kamu lucu sekali loh"""

# proses olah input
input_np = np.array([input])
input_tk = tokenizer.texts_to_sequences(input_np)
input_pd = pad_sequences(input_tk)

# hasil
output = model.predict(input_pd)
# output = model.predict(padded_test[20])
prob = np.round(np.max(output)*100, 2)
label = np.argmax(output)
actual_label = labels[label]

# print the label and the probability
print("Probalilitas: ", prob, "%")
print("Label sebenarnya: ", actual_label)

# print(padded_test)

1/1 [==============================] - 0s 436ms/step
Probalilitas:  97.86 %
Label sebenarnya:  Drama
